# Gesetze von Monaden

```haskell

return x >>= f      == f x
akt >>= return      == akt
(akt >>= f) >>= g   == akt >>= \x -> f x >>= g

## Für die KLausur dann
- erstelle <*> mit bind oder sowas
- AUSWENDIG LERNEN!!


```
Functor
   ^
   |
Applicative
   ^
   |
Monad

# Applicative für die Liste

```haskell
instance Applicative [] where
    pure :: a -> [a]
    pure x = [x]

    (<*>) :: [a -> b] -> [a] -> [b]
    [] <*> xs = []
    (f:fs) <*> xs = fmap f xs ++ (fs <*> xs)

instance Monad [] where
    return :: a -> [a]
    return = pure

    (>>=) :: [a] -> (a -> [b]) -> [b]
    [] >>= f = []
    (x:xs) >>= f = f x ++ (xs >>= f)

# Monad gesetze 
## 1
```haskell
    return x >>= f == fx
= [x] >>= f
= f x ++ ([] >>= f)
= f x ++ []
= f x
```

## 2
- mit fallunterscheidung, da xs auch die leere liste sein kann
- und dann mit struktureller induktion mit Annahme für Teilliste
```haskell
    xs >>= return == xs
case xs of
    [] -> xs >>= return
=   [] >>= return
=   = xs
    (x:xs) >>= return
= return x ++ (xs >>= return)
= [x] ++ xs
```

## 3
```haskell
    (xs >>= f) >>= g   == xs >>= \x -> f x >>= g
case xs of
    [] -> ([] >>= f) >>= g 
=   [] >>= g
=   []
=   [] >>= \x -> (f >>= g)

    (x:xs) -> 

```haskell
(>>) :: Monad m => m a -> m b -> m b
a1 >> a2 = a1 >>= \_ - a2

---

# jezt machen wir was mit bind

In [2]:
list = [1,2,3] >>= \x -> [x , x+1]

list

[1,2,2,3,3,4]

In [3]:
list = do
    x <- [1,2,3]
    [x, x + 1]

list

[1,2,2,3,3,4]

In [4]:
list = [1,2,3,4] >>= \x -> [3,4,5] >>= \y -> return (x,y)

list

[(1,3),(1,4),(1,5),(2,3),(2,4),(2,5),(3,3),(3,4),(3,5),(4,3),(4,4),(4,5)]

In [5]:
list = do
    x <- [1,2,3,4]
    y <- [3,4,5]
    return (x,y)
    
list

[(1,3),(1,4),(1,5),(2,3),(2,4),(2,5),(3,3),(3,4),(3,5),(4,3),(4,4),(4,5)]

In [6]:
list = [ (x,y) | x <- [1,2,3,4], y <- [3,4,5] ]

list

[(1,3),(1,4),(1,5),(2,3),(2,4),(2,5),(3,3),(3,4),(3,5),(4,3),(4,4),(4,5)]

In [7]:
list = do
    x <- [1,2,3,4]
    y <- [3,4,5]
    if x /= y then
        return (x,y)
    else 
        []

## das jetzt mit einer externen funktion

In [8]:
list = do
    x <- [1,2,3,4]
    y <- [3,4,5]
    guardList (x/=y)
    return (x,y)

guardList :: Bool -> [()]
guardList True = return ()
guardList False = []

list

[(1,3),(1,4),(1,5),(2,3),(2,4),(2,5),(3,4),(3,5),(4,3),(4,5)]

## wir schauen das guard mal an

In [9]:
import Control.Monad

:t guard

guard :: forall (f :: * -> *). Alternative f => Bool -> f ()

## festellungen
1. Listen sind Monaden mit return und (>>=)
2. Listen sind auch Monoide mit mempty ([]) und mappend (++)
3. beides ist in der Klasse MonadPlus zusammengeführt

```haskell
class Monad m => MonadPlus m where
    mzero :: m ()

    mplus :: m a -> m a -> m a

instance MonadPlus [] where
    mzero = []
    mplus = (++)

guard :: MonadPlus m => Bool -> m ()
guard True  = return ()
guard False = mzero

## Bild von oben erweitern

```
    Funktor
       |
  Applicative -
       |        \
     Monad    Alternative
        \     /
        MonadPlus
```


In [10]:
import Control.Applicative

:i Alternative

type Alternative :: (* -> *) -> Constraint
class Applicative f => Alternative f where
  empty :: f a
  (<|>) :: f a -> f a -> f a
  some :: f a -> f [a]
  many :: f a -> f [a]
  {-# MINIMAL empty, (<|>) #-}
  	-- Defined in ‘GHC.Base’
instance Alternative ZipList -- Defined in ‘Control.Applicative’
instance MonadPlus m => Alternative (WrappedMonad m) -- Defined in ‘Control.Applicative’
instance (Control.Arrow.ArrowZero a, Control.Arrow.ArrowPlus a) => Alternative (WrappedArrow a b) -- Defined in ‘Control.Applicative’
instance Alternative [] -- Defined in ‘GHC.Base’
instance Alternative Maybe -- Defined in ‘GHC.Base’
instance Alternative IO -- Defined in ‘GHC.Base’

## zu meiner frage wegen der verwirrung um Funktor ohne ein neutrales element

- ein Kontenmarkierter baum als Funtor ist einfaach
- applicative ist nicht möglich und Monade auch nicht

---

## Monad Instanz für Maybe

```haskell

return = Just
Nothing >>= _ = Nothing
(Just x) >>= f = f x

instance MonadPlus Maybe where
    mzero = Nothing
    mplus Nothing my = my
    mplus (Just x) my = Just x

```
Das `Just x` ist irgendwie seltsam

## Noch ein Gesetz für MonadPlus
was auf jeden fall gilt sind die Monoid gesetze
```haskell
mzero `mplus` a = a
a `mplus` mzero = a
(a `mplus` b) `mplus` c = a `mplus` (b `mplus` c)
```
jetzt noch ein distributivgesetz `x * (y + z) = xy + xz`
```haskell
(a `mplus` b) >>= f = a >>= f `mplus` b >>= f
```
Gesetze  gelten für die Listeninstanze

### einmal versuchen

In [11]:
[1,2] `mplus` [3,4] >>= \x -> return (x+1)

[2,3,4,5]

In [12]:
([1,2] >>= \x -> return (x+1)) `mplus` ([3,4] >>= \x -> return (x+1))

[2,3,4,5]

## gegenbeispiel von der Maybe instanz, dass die gesetze nicht gelten

```haskell
    Just False `mplus` Just True >>= guard
=   Just False >>= guard
=   mzero
=   Nothing

ABER

```haskell
    Just False >>= guard `mplus` Just True >>= guard
=   Nothing              `mplus` Just ()
=   Just ()

## Distributivgesetz gilt für Alternative und hier erfúllt die Maybe-Instanz das Distributivgesetz
Dies zeit den Unterschied in der Ausdrucksstärke zwischen Monad und Applicative

Alternative
```haskell
( cf <|> cg ) <*> cx = cf <*> cx <|> cg <*> cx
cx <*> ( cf <|> cg ) = cf <*> cx <|> cg <*> cx

---

# Jetzt schreiben wir ein Programm

In [13]:
data Tree a = Node (Tree a) a (Tree a)
                | Empty
        deriving (Show)

In [14]:
testTree = Node (Node Empty 42 Empty) 73 (Node Empty 55 Empty)


-- numberTree soll liefern: Node (Node Empty (1, 42) Empty) (2, 73) (Node Empty (3,55) Empty))
numberTree :: Tree a -> Tree (Int, a)
numberTree t = fst $ help t 0 where
    help :: Tree a -> Int -> (Tree (Int, a), Int)
    help Empty x    = (Empty, x)
    help  (Node l c r) x = 
        let (l', x1) = help l x 
            (r', x2) = help r $ x1 + 1
        in
            (Node l' (x1,c) r', x2)

numberTree testTree

Node (Node Empty (0,42) Empty) (1,73) (Node Empty (2,55) Empty)

## jetzt die state monade, welche dann eine globale variable einfach in das bind reinpackt


In [18]:
newtype State a = St (Int -> (a,Int))

numberTree t = runState 0 $ help t where 
    help :: Tree a -> State (Tree (Int, a))
    help Empty = return Empty
    help (Node l c r) = do
        l' <- help l
        n <- get
        put $ n+1
        r' <- help r
        return (Node l' (n, c) r')


instance Applicative State where
    pure = return
    (<*>) = ap

instance Functor State where
    fmap = (<$>)

instance Monad State where
    return :: a -> State a
    return x = St $ \s -> (x, s)

    (>>=) :: State a -> (a -> State b) -> State b
    St sf >>= f = St $ \s -> 
        let (x,s1) = sf s
            St sg = f x in 
          sg s1

get :: State Int
get = St $ \s -> (s,s)

put :: Int -> State ()
put ns = St $ \_ -> ((), ns)

runState :: Int -> State a -> a
runState init (St sf) = 
    let (res, s_fin) = sf init in
        res

numberTree testTree

Line 23: Use tuple-section
Found:
\ s -> (x, s)
Why not:
(x,)Line 35: Use const
Found:
\ _ -> ((), ns)
Why not:
const ((), ns)

Node (Node Empty (0,42) Empty) (1,73) (Node Empty (2,55) Empty)